## **Data Reading**

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import * 

df = spark.read.format("parquet").load("abfss://bronze@storagetosty.dfs.core.windows.net/products")
display(df)

## **Column Transformation**

In [0]:
df = df.drop("_rescued_data")
display(df)

## **SQL Functions**

In [0]:
df.createOrReplaceTempView("products")

In [0]:
%sql
SELECT current_catalog();
SHOW CATALOGS;

In [0]:
%sql
CREATE OR REPLACE FUNCTION `databricks-catalog`.bronze.discount_func( p_price DOUBLE)
RETURNS DOUBLE
LANGUAGE SQL
RETURN p_price * 0.90

In [0]:
%sql
SELECT product_id, price, `databricks-catalog`.bronze.discount_func( price ) AS discounted_price
FROM products

In [0]:
df = df.withColumn("discounted_price", expr("`databricks-catalog`.bronze.discount_func(price)"))
display(df)

In [0]:
%sql
CREATE OR REPLACE FUNCTION `databricks-catalog`.bronze.brand_upper( p_brand STRING)
RETURNS STRING
LANGUAGE PYTHON
AS
$$
  return p_brand.upper()
$$

In [0]:
%sql
SELECT brand, `databricks-catalog`.bronze.brand_upper( brand ) FROM products

In [0]:
display(df)

In [0]:
df = df.withColumn("brand_upper", expr("`databricks-catalog`.bronze.brand_upper(brand)"))
display(df)

In [0]:
df.write.format("delta")\
        .mode("overwrite")\
        .save("abfss://silver@storagetosty.dfs.core.windows.net/products")

In [0]:
spark.read.format("delta")\
            .load("abfss://silver@storagetosty.dfs.core.windows.net/products")\
            .display()

In [0]:
%sql
CREATE TABLE IF NOT EXISTS `databricks-catalog`.silver.silver_products
USING DELTA
LOCATION 'abfss://silver@storagetosty.dfs.core.windows.net/products'

In [0]:
%sql
SELECT * FROM `databricks-catalog`.silver.silver_products